### Introduction
This code looks at a fuzzy logic control interface for three inputs (Angle, Velocity, and Current). The membership functions for each input are defined and then the simulation is created to defuzzify the output given an input.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import skfuzzy
from skfuzzy import control as ctrl
from skfuzzy import defuzzify

In [ ]:
n = 0.01
S_ang = np.arange(-30,30+n,n) #support set: angle
S_vel = np.arange(-60,60+n,n) #support set: velocity
S_cur = np.arange(-3,3+n,n) #support set: current

ang = ctrl.Antecedent(S_ang, 'ang')
vel = ctrl.Antecedent(S_vel, 'vel')
cnt = ctrl.Consequent(S_cur, 'cnt')


In [ ]:
#Angle
mu_a_pl = skfuzzy.trapmf(S_ang,[-10,20,30+n,30+n])
mu_a_nl = skfuzzy.trapmf(S_ang,[-30,-30,-20,10])

plt.plot(S_ang,mu_a_pl,label="Positive Large")
plt.plot(S_ang,mu_a_nl,label="Negative Large")
plt.title("Membership - Anglular Position")
plt.legend()
plt.show()

# Doing it with skfuzzy: 
ang['PL'] = skfuzzy.trapmf(S_ang,[-10,20,30+n,30+n])
ang['NL'] = skfuzzy.trapmf(S_ang,[-30,-30,-20,10])
# ang.view()

In [ ]:
#Velcoity
mu_v_pl = skfuzzy.trapmf(S_vel,[-20,40,60,60])
mu_v_nl = skfuzzy.trapmf(S_vel,[-60,-60,-40,20])

plt.plot(S_vel,mu_v_pl, label="Positive Large")
plt.plot(S_vel,mu_v_nl, label="Negative Large")
plt.title("Membership - Anglular Velocity")
plt.legend()
plt.show()

#Doing it with skfuzzy
vel['PL'] = skfuzzy.trapmf(S_vel,[-20,40,60,60])
vel['NL'] = skfuzzy.trapmf(S_vel,[-60,-60,-40,20])
# vel.view()

In [ ]:
#Current- Control Action
mu_c_pl = skfuzzy.trapmf(S_cur,[0,2,3,3])
mu_c_nc = skfuzzy.trimf(S_cur,[-2,0,2])
mu_c_nl = skfuzzy.trapmf(S_cur,[-3,-3,-2,0])

plt.plot(S_cur,mu_c_pl,label="Positive Large")
plt.plot(S_cur,mu_c_nc,label="No change")
plt.plot(S_cur,mu_c_nl,label="Negative Large")
plt.title("Membership - Control Inference")
plt.legend()
plt.show()

#Doing it with skfuzzy
cnt['PL'] = skfuzzy.trapmf(S_cur,[0,2,3,3])
cnt['NC'] = skfuzzy.trimf(S_cur,[-2,0,2])
cnt['NL'] = skfuzzy.trapmf(S_cur,[-3,-3,-2,0])
# cnt.view()

### Fuzzy Logic Rules
For the simulation, these rules are defined. Then, an input is given (Angle, Velocity) ad the Current is inferred.

In [ ]:
rule1 = ctrl.Rule(ang['PL'] & vel['PL'], cnt['NL'])
rule2 = ctrl.Rule(ang['PL'] & vel['NL'], cnt['NC'])
rule3 = ctrl.Rule(ang['NL'] & vel['PL'], cnt['NC'])
rule4 = ctrl.Rule(ang['NL'] & vel['NL'], cnt['PL'])

rules = [rule1,rule2,rule3,rule4]
system = ctrl.ControlSystem(rules)
simulation = ctrl.ControlSystemSimulation(system)

#Angle=5, Velocity=15
simulation.input['ang']=5
simulation.input['vel']=15
simulation.compute()
simulation.output['cnt']